### 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import time
from glob import glob
from tqdm import tqdm
from scipy import interpolate

In [ ]:
# conda install tensorflow

In [ ]:
# conda install python=3.7

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

CNN으로 전처리해볼 것
1. 유량 3개, 수위 4개
2. 팔당댐 데이터

### 전체 수위 데이터셋 구축

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 불러오기 
전체df = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv')
# for colab
# /content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv
# for jupyter
# /Volumes/GoogleDrive/.shortcut-targets-by-id/1rUx7_7LmBqAQTbWEWVTGvw0_d0v6N2ZT/신입기수 프로젝트/Dataset/rawdata.csv

In [ ]:
수위 = 전체df[['청담대교_수위', '한강대교_수위', '잠수교_수위', '행주대교_수위']]

In [ ]:
수위

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,310.7,290.0,300.2,275.3
1,314.7,290.0,300.2,275.3
2,313.7,290.0,301.2,275.3
3,311.7,290.0,301.2,276.3
4,311.7,291.0,301.2,277.3
...,...,...,...,...
276331,0.0,0.0,0.0,0.0
276332,0.0,0.0,0.0,0.0
276333,0.0,0.0,0.0,0.0
276334,0.0,0.0,0.0,0.0


In [ ]:
nanIndex = 수위[수위.청담대교_수위.isnull()].index

In [ ]:
nanIndex

Int64Index([140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620,
            140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628,
            140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636,
            140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644,
            140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652,
            140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660,
            140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668,
            140669, 140670, 236080],
           dtype='int64')

In [ ]:
consecutive = list(nanIndex)[:-1]
unconsecutive = list(nanIndex)[-1]
print(consecutive)
print(unconsecutive)

[140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620, 140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628, 140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636, 140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644, 140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652, 140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660, 140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668, 140669, 140670]
236080


In [ ]:
# 58개 결측치 앞 인덱스까지 인풋 생성
x = 수위.loc[:140612]

# 결측치로 아웃풋 생성
null = 수위[140613:140671]

In [ ]:
null

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
140613,NaN,NaN,NaN,NaN
140614,NaN,NaN,NaN,NaN
140615,NaN,NaN,NaN,NaN
140616,NaN,NaN,NaN,NaN
140617,NaN,NaN,NaN,NaN
140618,NaN,NaN,NaN,NaN
140619,NaN,NaN,NaN,NaN
140620,NaN,NaN,NaN,NaN
140621,NaN,NaN,NaN,NaN
140622,NaN,NaN,NaN,NaN


In [ ]:
x

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,310.7,290.0,300.2,275.3
1,314.7,290.0,300.2,275.3
2,313.7,290.0,301.2,275.3
3,311.7,290.0,301.2,276.3
4,311.7,291.0,301.2,277.3
...,...,...,...,...
140608,435.7,410.0,415.2,374.3
140609,438.7,413.0,418.2,375.3
140610,437.7,413.0,418.2,373.3
140611,434.7,413.0,415.2,373.3


In [ ]:
# conda install scikit-learn

In [ ]:
# train, test data 생성할 때 필요
from sklearn.model_selection import train_test_split
# 정규화할 때 필요
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [ ]:
# input 정규화
x_regular = x.copy()

for i in range(len(x.columns)):
    scaler = StandardScaler()                       #StandardScaler() , MinMaxScaler() , RobustScaler()
    a=np.array(x_regular.iloc[:,i])
    tmp_x = scaler.fit_transform(a.reshape(-1,1))
    x_regular.iloc[:,i] = tmp_x

In [ ]:
x_regular

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,-0.213183,-0.292141,-0.168958,-0.473340
1,-0.150481,-0.292141,-0.168958,-0.473340
2,-0.166156,-0.292141,-0.151149,-0.473340
3,-0.197507,-0.292141,-0.151149,-0.450834
4,-0.197507,-0.273710,-0.151149,-0.428328
...,...,...,...,...
140608,1.746252,1.919546,1.879052,1.754767
140609,1.793278,1.974838,1.932478,1.777274
140610,1.777603,1.974838,1.932478,1.732261
140611,1.730576,1.974838,1.879052,1.732261


### 청담대교 수위

In [ ]:
start = time.time()

y_predict = []

xtrain = []
ytrain = []

w =  432 # window size
n = 58 # 예측할 데이터 개수

# xtrain 생성
# for i in range(len(x_regular)-w):
#     xtrain.append(np.array(x_regular.iloc[i:i + w, :]).astype(float))

for i in tqdm(range(len(x_regular)-w-n)):
    xtrain.append(np.array(x_regular.iloc[i:i + w, 0]).astype(float))

100%|██████████| 140123/140123 [00:17<00:00, 8088.92it/s]


In [ ]:
# ytrain 생성
# for i in range(w, len(x_regular)):
#     ytrain.append(np.array(x_regular.iloc[i, :]).astype(float))
ytrain = []

for i in tqdm(range(w, len(x_regular)-n)):
  ytrain.append(np.array(x_regular.iloc[i:i+n, 0]).astype(float))

100%|██████████| 140123/140123 [00:14<00:00, 9594.30it/s] 


In [ ]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)
print(xtrain.shape)
print(ytrain.shape)

(140123, 432)
(140123, 58)


In [ ]:
df_xtrain = pd.DataFrame(xtrain, columns = [i for i in range(432)])

In [ ]:
df_ytrain = pd.DataFrame(ytrain, columns = [f'R{i}' for i in range(58)])

In [ ]:
df_series = pd.concat([df_xtrain, df_ytrain], axis = 1)

In [ ]:
df_series.shape

(140123, 490)

In [ ]:
xtrain.reshape(140123, 432, 1)
ytrain.reshape(140123, 58, 1)

array([[[-0.30723539],
        [-0.27588445],
        [-0.32291086],
        ...,
        [-0.18183161],
        [-0.15048066],
        [-0.15048066]],

       [[-0.27588445],
        [-0.32291086],
        [-0.30723539],
        ...,
        [-0.15048066],
        [-0.15048066],
        [-0.15048066]],

       [[-0.32291086],
        [-0.30723539],
        [-0.36993728],
        ...,
        [-0.15048066],
        [-0.15048066],
        [-0.10345424]],

       ...,

       [[ 0.05330049],
        [ 0.02194954],
        [-0.02507688],
        ...,
        [ 1.68354968],
        [ 1.74625158],
        [ 1.79327799]],

       [[ 0.02194954],
        [-0.02507688],
        [-0.0721033 ],
        ...,
        [ 1.74625158],
        [ 1.79327799],
        [ 1.77760252]],

       [[-0.02507688],
        [-0.0721033 ],
        [-0.10345424],
        ...,
        [ 1.79327799],
        [ 1.77760252],
        [ 1.7305761 ]]])

In [ ]:
# label
labels_col = [f'R{i}' for i in range(58)]
labels = df_series[labels_col]
series = df_series.drop(labels_col, axis=1)
labels.values.shape

(140123, 58)

In [ ]:
# x_train, x_val, y_train, y_val 분류
x_train, x_val, y_train, y_val = train_test_split(series, labels.values, test_size = 0.4, random_state = 0)

In [ ]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,422,423,424,425,426,427,428,429,430,431
133922,-0.354262,-0.354262,-0.369937,-0.385613,-0.432639,-0.463990,-0.479666,-0.511017,-0.526692,-0.526692,...,0.037625,0.100327,0.131678,0.131678,0.116002,0.084651,0.053300,-0.025077,-0.087779,-0.181832
32870,-0.495341,-0.511017,-0.526692,-0.526692,-0.542367,-0.542367,-0.558043,-0.558043,-0.558043,-0.558043,...,-0.558043,-0.558043,-0.558043,-0.558043,-0.573718,-0.573718,-0.589394,-0.605069,-0.605069,-0.605069
53652,-0.291560,-0.354262,-0.448315,-0.511017,-0.495341,-0.463990,-0.432639,-0.416964,-0.385613,-0.369937,...,-0.761824,-0.730473,-0.730473,-0.714798,-0.746149,-0.730473,-0.746149,-0.730473,-0.746149,-0.730473
2311,-0.542367,-0.542367,-0.542367,-0.542367,-0.542367,-0.526692,-0.511017,-0.526692,-0.542367,-0.526692,...,-0.385613,-0.401288,-0.385613,-0.401288,-0.416964,-0.432639,-0.495341,-0.495341,-0.511017,-0.511017
132218,-0.636420,-0.620745,-0.620745,-0.620745,-0.636420,-0.652096,-0.652096,-0.652096,-0.667771,-0.683447,...,-0.558043,-0.589394,-0.589394,-0.573718,-0.542367,-0.526692,-0.511017,-0.511017,-0.511017,-0.526692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,-0.009401,0.006274,0.006274,0.021950,0.021950,0.037625,0.037625,0.037625,0.037625,0.037625,...,0.257082,0.225731,0.147353,0.147353,0.084651,0.084651,0.068976,0.006274,0.021950,0.006274
97639,1.150584,1.072206,0.962478,0.852750,0.727346,0.586267,0.476538,0.429512,0.398161,0.366810,...,-0.871552,-0.887228,-0.871552,-0.871552,-0.871552,-0.871552,-0.871552,-0.871552,-0.871552,-0.871552
95939,-0.699122,-0.714798,-0.730473,-0.730473,-0.730473,-0.714798,-0.699122,-0.683447,-0.699122,-0.683447,...,-0.840201,-0.855877,-0.855877,-0.871552,-0.855877,-0.855877,-0.855877,-0.840201,-0.808851,-0.746149
117952,0.335459,0.319784,0.304108,0.288433,0.257082,0.257082,0.225731,0.225731,0.194380,0.178704,...,1.134908,1.150584,1.134908,1.119233,1.056531,0.978153,0.868425,0.774372,0.680319,0.617618


In [ ]:
# xtest = 

In [ ]:
# 모델링 및 모델 학습

if 'model' in locals(): 
    del model 
if 'history' in locals():
    del history


#He_normal
He_normal = keras.initializers.he_normal(seed=None)

model = Sequential()
model.add(Conv1D(filters = 8, input_shape = (w, 1), kernel_size = 2, padding='causal', dilation_rate=2, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))  #0.2~0.5  0.8까지

model.add(Conv1D(filters = 16, kernel_size = 2, padding='causal', dilation_rate=4, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters = 32, kernel_size = 1))

model.add(Flatten())   #model.add(GlobalAveragePooling1D(pool_size=2))

model.add(Dense(50, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(58))


def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

opt = tf.keras.optimizers.Adam(lr=0.005) #기본값 0.001
model.compile(optimizer=opt, loss=root_mean_squared_error, metrics =["accuracy"])
model.summary()



callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5),
                keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                                monitor='val_loss',
                                                save_best_only=True)]



history = model.fit(xtrain, ytrain, epochs=300, validation_data = (x_val, y_val), verbose=1, callbacks=callbacks, batch_size = 3742) #batch size..?


keras_model_best = keras.models.load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

keras_model_best.evaluate(x_val,y_val)

# y_predict.append(keras_model_best.predict(xtest))

print("time :", time.time() - start, "초")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 432, 8)            24        
                                                                 
 batch_normalization (BatchN  (None, 432, 8)           32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 432, 8)            0         
                                                                 
 dropout (Dropout)           (None, 432, 8)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 432, 16)           272       
                                                                 
 batch_normalization_1 (Batc  (None, 432, 16)          64        
 hNormalization)                                        

In [ ]:
x_test = x['청담대교_수위'][-432:]

In [ ]:
x_test = x_test.reshape((1, 432, 1))

In [ ]:
x_test.shape

(1, 432, 1)

In [ ]:
y_predict = keras_model_best.predict(x_test)

In [ ]:
y_predict.shape

(1, 58)

In [ ]:
pd.Series(list(y_predict))

0    [356.7141, 353.11176, 363.21722, 363.84454, 36...
dtype: object

In [ ]:
수위.loc[consecutive, '청담대교_수위'] = y_predict

In [ ]:
수위.loc[consecutive, '청담대교_수위']

140613    356.714111
140614    353.111755
140615    363.217224
140616    363.844543
140617    365.909119
140618    363.827393
140619    364.425659
140620    364.011261
140621    363.490387
140622    364.510864
140623    364.045715
140624    363.996552
140625    361.002045
140626    359.652863
140627    360.933014
140628    355.186127
140629    353.749451
140630    354.733490
140631    350.926605
140632    350.105225
140633    347.467255
140634    344.468597
140635    343.386444
140636    339.775391
140637    336.228516
140638    329.561920
140639    326.070129
140640    325.695068
140641    318.948090
140642    316.603027
140643    314.934814
140644    311.850098
140645    311.948181
140646    312.573517
140647    310.761322
140648    313.600708
140649    314.994293
140650    315.581421
140651    317.602631
140652    321.123718
140653    321.627136
140654    323.130249
140655    320.198242
140656    324.375580
140657    324.597595
140658    325.360321
140659    323.838043
140660    324

In [ ]:
# unconsecutive 부분 보간

unconsecutive2 = [unconsecutive-1, unconsecutive, unconsecutive+1]
수위.loc[unconsecutive2, '청담대교_수위'] = 수위.loc[unconsecutive2, '청담대교_수위'].interpolate()

In [ ]:
수위['청담대교_수위'].isnull().sum()

0

In [ ]:
수위['청담대교_수위'].to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/CNN_전처리/청담대교_수위.csv', index = False, encoding = 'cp949')